In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

df = pd.read_csv('LabeledScoredTweets.csv')
df = df.drop(['Unnamed: 0'],axis=1)


In [2]:
#!pip3 install vaderSentiment

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#replace comments_df with the current dataframe using
analyzer = SentimentIntensityAnalyzer()
df['polarity_scores'] = df['Content'].apply(lambda x: analyzer.polarity_scores(x))

df['compound_score'] = df['polarity_scores'].apply(lambda score_dict: score_dict['compound'])

df = df.drop(['polarity_scores'], axis=1)
df

,Date,Content,Username,Target,Score,compound_score
0,2023-03-10,Is Fed Chair Powell “willing to take a recessi...,YahooFinance,Down,-12,-0.8819
1,2023-03-24,"JPMORGAN SHARES DOWN 1.1%, MORGAN STANLEY DOWN...",FirstSquawk,Up,-12,0.6874
2,2023-04-19,SHARES OF CRYPTO COS DOWN AFTER BITCOIN NEARS ...,FirstSquawk,Up,-10,-0.3400
3,2023-03-24,"⚠️ COINBASE GLOBAL SHARES DOWN 3.6%, BITFARMS...",PriapusIQ,Up,-10,-0.3400
4,2023-04-13,"FOMC: ""The Fed staff now expects the economy t...",zerohedge,Up,-10,-0.8885
...,...,...,...,...,...,...
32427,2023-03-20,SHARES OF MAJOR U.S. BANKS UP\n\nSHARES OF GOL...,FirstSquawk,Up,13,0.9282
32428,2023-04-13,SHARES OF CRYPTO STOCKS UP AFTER BITCOIN CLIMB...,DeItaone,Up,14,0.1363
32429,2023-04-04,Soaring technology stocks run the risk of rall...,business,Down,14,0.8360
32430,2023-03-02,• Stocks open mixed this morning with sectors ...,YahooFinance,Up,15,-0.0772


In [4]:
#df.to_csv('VaterScoredTweets.csv')

In [5]:
binary_df = df.copy()
binary_df['bTarget'] = binary_df['bTarget'] = binary_df['Target'].map({'Up': 1, 'Down': 0})
biary_df = binary_df.filter(['Date','Target','Score'])
userName = binary_df.filter(['Username'])
userName = pd.get_dummies(userName)
binary_df = binary_df.join(userName)
binary_df = binary_df.drop(['Username','Target'],axis=1)
binary_df

,Date,Content,Score,compound_score,bTarget,Username_DeItaone,Username_FirstSquawk,Username_PriapusIQ,Username_Reuters,Username_WSJ,Username_YahooFinance,Username_business,Username_financialjuice,Username_unusual_whales,Username_zerohedge
0,2023-03-10,Is Fed Chair Powell “willing to take a recessi...,-12,-0.8819,0,0,0,0,0,0,1,0,0,0,0
1,2023-03-24,"JPMORGAN SHARES DOWN 1.1%, MORGAN STANLEY DOWN...",-12,0.6874,1,0,1,0,0,0,0,0,0,0,0
2,2023-04-19,SHARES OF CRYPTO COS DOWN AFTER BITCOIN NEARS ...,-10,-0.3400,1,0,1,0,0,0,0,0,0,0,0
3,2023-03-24,"⚠️ COINBASE GLOBAL SHARES DOWN 3.6%, BITFARMS...",-10,-0.3400,1,0,0,1,0,0,0,0,0,0,0
4,2023-04-13,"FOMC: ""The Fed staff now expects the economy t...",-10,-0.8885,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32427,2023-03-20,SHARES OF MAJOR U.S. BANKS UP\n\nSHARES OF GOL...,13,0.9282,1,0,1,0,0,0,0,0,0,0,0
32428,2023-04-13,SHARES OF CRYPTO STOCKS UP AFTER BITCOIN CLIMB...,14,0.1363,1,1,0,0,0,0,0,0,0,0,0
32429,2023-04-04,Soaring technology stocks run the risk of rall...,14,0.8360,0,0,0,0,0,0,0,1,0,0,0
32430,2023-03-02,• Stocks open mixed this morning with sectors ...,15,-0.0772,1,0,0,0,0,0,1,0,0,0,0


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [7]:
X = df.drop(['Target','Content','Date'],axis=1)
Y = df.filter(['Target'])

bX = binary_df.drop(['bTarget','Content','Date'],axis=1)
bY = binary_df.filter(['bTarget'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.4, random_state=2023)

bX_train, bX_test, by_train, by_test = train_test_split(bX,bY,test_size=0.4, random_state=2023)



nb = GaussianNB()
nb.fit(bX_train, by_train)
y_pred = nb.predict(bX_test)
accuracy = (y_pred == by_test.values.ravel()).sum() / len(by_test)
print(f"Guassian NB Accuracy: {accuracy:.6f}")



rf = RandomForestClassifier(random_state=2023)
rf.fit(bX_train, by_train)
y_pred = rf.predict(bX_test)
accuracy = (y_pred == by_test.values.ravel()).sum() / len(y_test)
print(f"Random Forest Accuracy: {accuracy:.6f}")


logreg = LogisticRegression(random_state=2023)
logreg.fit(bX_train, by_train)
y_pred = logreg.predict(bX_test)
accuracy = (y_pred == by_test.values.ravel()).sum() / len(by_test)
print(f"Logistic Regression Accuracy: {accuracy:.6f}")



tree = DecisionTreeClassifier(random_state=2023)
tree.fit(bX_train, by_train)
y_pred = tree.predict(bX_test)
accuracy = (y_pred == by_test.values.ravel()).sum() / len(y_test)
print(f"Decision Tree Accuracy: {accuracy:.6f}")

C:\Users\Temp\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Temp\AppData\Local\Temp\ipykernel_30624\739856651.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(bX_train, by_train)


Guassian NB Accuracy: 0.535420
Random Forest Accuracy: 0.526247
Logistic Regression Accuracy: 0.533724
Decision Tree Accuracy: 0.524243


C:\Users\Temp\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
